# ACXplainer DEMO

In [1]:
import shap 
import sys
from sklearn.model_selection import train_test_split
from acv_explainers import *
import numpy as np

sys.setrecursionlimit(10000)

In [2]:
import skranger
skranger.__version__

'0.6.1'

### Load data

In [2]:
X,y = shap.datasets.adult()
X_display,y_display = shap.datasets.adult(display=True)

# create a train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=7)

In [3]:
X_train.shape

(8140, 12)

### Train Explainer (ACXplainer)

In [4]:
ac_explainer = ACXplainer(classifier=True, n_estimators=5, max_depth=2)
ac_explainer.fit(X_train, y_train)

In [5]:
ac_explainer.predict(X_train)

array([False, False, False, ..., False,  True, False])

In [6]:
from sklearn.metrics import roc_auc_score

roc_auc_score(ac_explainer.predict(X_test), y_test)

0.8642037047987893

### Launch the webApp

In [7]:
# import acv_app
# import os

# # compile the ACXplainer
# acv_app.compile_ACXplainers(ac_explainer, X_train, y_train, X_test, y_test, path=os.getcwd())

# # Launch the webapp
# acv_app.run_webapp(pickle_path=os.getcwd())

### Compute the Minimal Sufficient Explanation

In [8]:
max_size = 10

sdp_importance, sdp_index, size, sdp = ac_explainer.importance_sdp_rf(X_train.iloc[:max_size], y_train[:max_size].astype(np.double),
                                                                     X_train, y_train.astype(np.double))

 71%|███████▏  | 5/7 [00:02<00:00,  2.27it/s]


In [8]:
idx = np.random.randint(0, max_size)
print('The Minimal Sufficient Explanation of obs {} is = {} and its has a SDP = {}'.format(idx, sdp_index[idx, :size[idx]], sdp[idx]))

The Minimal Sufficient Explanation of obs 0 is = [ 8  2  9  4 10] and its has a SDP = 0.8178302402579478


### Compute the Sufficient Rules

In [9]:
from acv_explainers.utils import get_active_null_coalition_list

S_star, N_star = get_active_null_coalition_list(sdp_index, size)

In [10]:
sdp, rules, sdp_all, rules_data, w = ac_explainer.compute_sdp_maxrules(X_train.iloc[:max_size], y_train[:max_size].astype(np.double),
                                                         X_train, y_train.astype(np.double), S_star, verbose=True)

100%|██████████| 10/10 [01:28<00:00,  8.86s/it]


- Note that the rules for an observation is a rectangle of dimension (# Features x 2) i.e an interval for each variable. Below, we plot the rule only on the relevant axis, the remaining axis have no rule or in other word have a rule equals to $\mathbb{R}$.

In [11]:
rule = rules[idx]
S = S_star[idx]
columns = [X_train.columns[i] for i in range(X_train.shape[1])]
rule_string = ['{} <= {} <= {}'.format(rule[i, 0] if rule[i, 0] > -1e+10 else -np.inf, columns[i],
                                               rule[i, 1] if rule[i, 1] < 1e+10 else +np.inf) for i in S]
rule_string = ' and '.join(rule_string)

print('The Minimal Sufficient Rule of obs {} is \n {}'.format(idx, rule_string))

The Minimal Sufficient Rule of obs 0 is 
 -inf <= Capital Gain <= 5095.5 and -inf <= Education-Num <= 12.5 and -inf <= Capital Loss <= 1793.5 and -inf <= Occupation <= 9.5 and -inf <= Hours per week <= inf


### Compute all the Sufficient Explanations

In [12]:
sufficient_expl, sdp_expl, sdp_global = ac_explainer.sufficient_expl_rf(X_train.iloc[:max_size], y_train[:max_size].astype(np.double),
                                                                     X_train, y_train.astype(np.double), pi_level=0.8)

100%|██████████| 7/7 [00:02<00:00,  2.84it/s]


In [13]:
print('The set of All the Sufficient explanations of obs {} is \n {}'.format(idx, sufficient_expl[idx][1:]))

The set of All the Sufficient explanations of obs 0 is 
 [[8, 2], [8, 9], [8, 4]]
